In [1]:
import sys
import logging

sys.path.insert(1, '../')

from importlib import reload

reload(logging)
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG, format='%(message)s')

In [2]:
from tsad.pipelines import Pipeline
from tsad.tasks.eda import HighLevelDatasetAnalysisTask, TimeDiscretizationTask, FindNaNTask, EquipmentDowntimeTask
from tsad.tasks.preprocess import ValueRangeProcessingTask, PrepareSeqSamplesTask

from tsad.base.datasets import load_combines

matplotlib data path: /Users/telepzk/Projects/tsad/venv/lib/python3.10/site-packages/matplotlib/mpl-data
CONFIGDIR=/Users/telepzk/.matplotlib
interactive is False
platform is darwin
CACHEDIR=/Users/telepzk/.matplotlib
Using fontManager instance from /Users/telepzk/.matplotlib/fontlist-v330.json


/Users/telepzk/Projects/tsad/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset = load_combines()

#### Working with Task without Pipeline
##### Работа с задачей без конвейера

In [4]:
cdt = HighLevelDatasetAnalysisTask()
cdt_df, cdt_result = cdt.fit_predict(dataset.frame)
cdt_result.show()

'Dataset size: 450, features: 4'

'Time index from 2023-04-21 13:32:48.228000 to 2023-04-25 23:59:59.999000'

'Total duration: 4 days 10:27:11.771000'

float64    4
dtype: int64

Описание
Anker     float64
Cut       float64
Go        float64
Uncert    float64
dtype: object

In [5]:
discretization = TimeDiscretizationTask()
discretization.com = cdt_result

_, discretization_result = discretization.fit_predict(cdt_df)

discretization_result.dataset_analysis_result = cdt_result
discretization_result.show()

['__abstractmethods__', '__annotations__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abc_impl', 'amount_unique_diff', 'amount_unique_diff_percent', 'dataset_analysis_result', 'freq_tobe', 'frequence_of_diff_interval', 'index_freq_climed', 'max_diff', 'min_diff', 'most_frequent_diff_amount_cases', 'most_frequent_diff_amount_cases_percent', 'most_frequent_diff_value', 'save', 'show']


'During the period from 2023-04-21 13:32:48.228000 to 2023-04-25 23:59:59.999000'

'With a total duration of 4 days 10:27:11.771000'

'Distribution of periods between points'

до 0 days 00:00:00                                  0
c 0 days 00:00:00 до 0 days 00:00:00.000000001      0
c 0 days 00:00:00.000000001 до 0 days 00:00:01    134
c 0 days 00:00:01 до 0 days 00:01:00              109
c 0 days 00:01:00 до 0 days 01:00:00              202
c 0 days 01:00:00 до 0 days 08:00:00                3
c 0 days 08:00:00 до 1 days 00:00:00                0
c 1 days 00:00:00 до 7 days 00:00:00                1
c 7 days 00:00:00 до 30 days 00:00:00               0
от 30 days 00:00:00                                 0
dtype: int64

...

'The most frequently period 0 days 00:00:00.219000'

'The number and proportion of the most frequently periods is 20                 , which is 4.444 %'

'The number of unique periods 329 out of 450                points in the dataset, which is 73.111%'

'Minimum period: 0 days 00:00:00.002000, Maximum period: 3 days 09:45:32.950000'

'SELECTED PERIOD for RESAMPLING: 10ms'

#### Working with separate pipelines for group of tasks
##### Работа с отдельными конвейерами для группы задач

In [6]:
eda_tasks = [
    HighLevelDatasetAnalysisTask(),
    TimeDiscretizationTask(freq_tobe='1s'),
    FindNaNTask(),
    EquipmentDowntimeTask()
]

eda_pipeline = Pipeline(eda_tasks, show=False)
eda_fit_df = eda_pipeline.fit_predict(dataset.frame)

In [7]:
preprocess_tasks = [
    ValueRangeProcessingTask(),
    PrepareSeqSamplesTask()
]

preprocess_pipeline = Pipeline(preprocess_tasks, results=eda_pipeline.results, show=True)
preprocess_fit_df = preprocess_pipeline.fit_predict(eda_fit_df)
preprocess_pipeline.predict(eda_fit_df)

Values outside the allowed range will be replaced with NaN
Values outside the allowed range will be replaced with NaN


[[Описание                 Anker  Cut   Go  Uncert
  Время                                           
  2023-04-21 13:32:48.228    0.0  NaN  NaN     NaN
  2023-04-21 13:32:48.230    NaN  NaN  0.0     NaN
  2023-04-24 23:18:21.180    NaN  1.0  NaN     0.0
  2023-04-25 00:44:54.778    NaN  0.0  NaN     1.0
  2023-04-25 00:44:54.997    NaN  1.0  NaN     0.0
  2023-04-25 00:45:55.130    NaN  0.0  NaN     1.0
  2023-04-25 00:45:55.380    NaN  1.0  NaN     0.0
  2023-04-25 00:49:55.709    NaN  0.0  NaN     1.0
  2023-04-25 00:49:55.990    NaN  1.0  NaN     0.0
  2023-04-25 00:55:56.304    NaN  0.0  NaN     1.0,
  Описание                 Anker  Cut   Go  Uncert
  Время                                           
  2023-04-21 13:32:48.230    NaN  NaN  0.0     NaN
  2023-04-24 23:18:21.180    NaN  1.0  NaN     0.0
  2023-04-25 00:44:54.778    NaN  0.0  NaN     1.0
  2023-04-25 00:44:54.997    NaN  1.0  NaN     0.0
  2023-04-25 00:45:55.130    NaN  0.0  NaN     1.0
  2023-04-25 00:45:55.380    N

#### Work with Pipeline
##### Работа с конвейером

In [8]:
pipeline = Pipeline(eda_tasks + preprocess_tasks)
pipeline.fit_predict(dataset.frame)

Values outside the allowed range will be replaced with NaN


[[Описание                 Anker  Cut   Go  Uncert
  Время                                           
  2023-04-21 13:32:48.228    0.0  NaN  NaN     NaN
  2023-04-21 13:32:48.230    NaN  NaN  0.0     NaN
  2023-04-24 23:18:21.180    NaN  1.0  NaN     0.0
  2023-04-25 00:44:54.778    NaN  0.0  NaN     1.0
  2023-04-25 00:44:54.997    NaN  1.0  NaN     0.0
  2023-04-25 00:45:55.130    NaN  0.0  NaN     1.0
  2023-04-25 00:45:55.380    NaN  1.0  NaN     0.0
  2023-04-25 00:49:55.709    NaN  0.0  NaN     1.0
  2023-04-25 00:49:55.990    NaN  1.0  NaN     0.0
  2023-04-25 00:55:56.304    NaN  0.0  NaN     1.0,
  Описание                 Anker  Cut   Go  Uncert
  Время                                           
  2023-04-21 13:32:48.230    NaN  NaN  0.0     NaN
  2023-04-24 23:18:21.180    NaN  1.0  NaN     0.0
  2023-04-25 00:44:54.778    NaN  0.0  NaN     1.0
  2023-04-25 00:44:54.997    NaN  1.0  NaN     0.0
  2023-04-25 00:45:55.130    NaN  0.0  NaN     1.0
  2023-04-25 00:45:55.380    N